## Feature Extraction

In [11]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm

data_dir = r"C:\Users\sneha\PycharmProjects\DermaScan\rotated_dataset"

def extract_features(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256, 256))

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # normalized features (histogram)
        hist_b = cv2.calcHist([img], [0], None, [32], [0, 256]).flatten()
        hist_g = cv2.calcHist([img], [1], None, [32], [0, 256]).flatten()
        hist_r = cv2.calcHist([img], [2], None, [32], [0,256]).flatten()
        color_features = np.concatenate([hist_b, hist_g, hist_r])
        color_features = color_features / np.sum(color_features)

        # texture features (GLCM)
        glcm = graycomatrix(gray, distances = [5], angles = [0], levels = 256, symmetric = True, normed = True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        texture_features = [contrast, dissimilarity, homogeneity, energy, correlation]

        # edge features (canny edge count)
        edges = cv2.Canny(gray, 100, 200)
        edge_density = np.sum(edges) / (256*256)

        # combining features
        features = np.concatenate([color_features, texture_features, [edge_density]])
        return features
    except Exception as e:
        print("Error processing:", img_path, e)
        return None

features_list = []
labels = []

for cls in os.listdir(data_dir):
    cls_path = os.path.join(data_dir, cls)
    if not os.path.isdir(cls_path):
        continue
    for img_name in tqdm(os.listdir(cls_path), desc = f"Extracting {cls}"):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(cls_path, img_name)
            feat = extract_features(img_path)
            if feat is not None:
                features_list.append(feat)
                labels.append(cls)

df = pd.DataFrame(features_list)
df['labels'] = labels

df.to_csv("skin_features.csv", index=False)
print("Features extracted successfully !!!")

Extracting wrinkles: 100%|██████████| 250/250 [00:01<00:00, 146.42it/s]


Features extracted successfully !!!


## Train test split & Evaluation metrics

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import joblib

df = pd.read_csv("skin_features.csv")

X = df.drop("labels", axis=1)
y = df["labels"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

joblib.dump(model, "dermascan_rf_model.joblib")
print("Model trained & saved successfully!")


Accuracy: 0.9955555555555555

Classification Report:
               precision    recall  f1-score   support

        acne       1.00      1.00      1.00        50
  dark_spots       1.00      1.00      1.00        50
         dry       0.96      1.00      0.98        50
       milia       1.00      1.00      1.00        50
        oily       1.00      1.00      1.00        50
pigmentation       1.00      1.00      1.00        50
       pores       1.00      0.96      0.98        50
     redness       1.00      1.00      1.00        50
    wrinkles       1.00      1.00      1.00        50

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450

Model trained & saved successfully!


## Testing & Prediction

In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm
import joblib

def extract_features(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256, 256))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)

        # color histogram
        hist_b = cv2.calcHist([img], [0], None, [32], [0, 256]).flatten()
        hist_g = cv2.calcHist([img], [1], None, [32], [0, 256]).flatten()
        hist_r = cv2.calcHist([img], [2], None, [32], [0, 256]).flatten()
        color_features = np.concatenate([hist_b, hist_g, hist_r])
        color_features = color_features / np.sum(color_features)

        # texture (GLCM)
        glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        texture_features = [contrast, dissimilarity, homogeneity, energy, correlation]

        # edge feature
        edges = cv2.Canny(gray, 100, 200)
        edge_density = np.sum(edges) / (256*256)

        features = np.concatenate([color_features, texture_features, [edge_density]])
        return features
    except Exception as e:
        print("Error processing:", img_path, e)
        return None

model = joblib.load("dermascan_rf_model.joblib")

test_folder = r"C:\Users\sneha\PycharmProjects\DermaScan\new_images"

test_features = []
image_names = []

for img_name in tqdm(os.listdir(test_folder)):
    img_path = os.path.join(test_folder, img_name)
    features = extract_features(img_path)
    if features is not None:
        test_features.append(features)
        image_names.append(img_name)

X_test = pd.DataFrame(test_features)

predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)

classes = model.classes_  # to map probabilities with class names
threshold = 10  # show all conditions with >10% probability
for name, prob in zip(image_names, probabilities):
    print(f"\n{name} → Probable skin conditions:")
    for cls, p in zip(model.classes_, prob):
        if p * 100 >= threshold:
            print(f"   {cls}: {p*100:.2f}%")



100%|██████████| 5/5 [00:00<00:00, 36.57it/s]


WhatsApp Image 2025-11-09 at 3.57.04 PM (1).jpeg → Probable skin conditions:
   acne: 10.50%
   oily: 16.50%
   pigmentation: 15.50%
   redness: 22.00%
   wrinkles: 15.00%

WhatsApp Image 2025-11-09 at 3.57.04 PM.jpeg → Probable skin conditions:
   dark_spots: 10.00%
   milia: 16.50%
   oily: 12.00%
   pigmentation: 15.00%
   redness: 13.00%
   wrinkles: 12.50%

WhatsApp Image 2025-11-09 at 3.57.05 PM.jpeg → Probable skin conditions:
   dark_spots: 14.50%
   oily: 25.50%
   pigmentation: 22.50%

WhatsApp Image 2025-11-09 at 5.03.11 PM.jpeg → Probable skin conditions:
   acne: 10.50%
   dry: 13.00%
   milia: 11.00%
   oily: 10.50%
   pigmentation: 12.50%
   wrinkles: 24.00%

WhatsApp Image 2025-11-09 at 5.03.25 PM.jpeg → Probable skin conditions:
   dark_spots: 11.00%
   dry: 11.00%
   milia: 18.00%
   oily: 11.00%
   pigmentation: 10.00%
   redness: 13.00%
   wrinkles: 15.00%



C:\Users\sneha\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\sneha\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


*** ⚠️ Note: Results may slightly vary due to lighting, camera angle, or image quality. For best accuracy, take the photo in natural light without flash or filters. ***

''''
from flask import Flask, render_template, request
import pandas as pd
import os

app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return render_template('form.html')

@app.route('/submit', methods=['POST'])
def submit():
    # Collecting data from form
    data = {
        'age': request.form['age'],
        'gender': request.form['gender'],
        'skin_type': request.form['skin-type'],
        'sensitive': request.form['sensitive'],
        'acne': request.form['acne'],
        'pigmentation': request.form['pigmentation'],
        'wrinkles': request.form['wrinkles'],
        'dark_spots': request.form['dark-spots'],
        'whiteheads': request.form['whiteheads'],
        'blackheads': request.form['blackheads'],
        'oiliness': request.form['oiliness'],
        'dryness': request.form['dryness'],
        'redness': request.form['redness'],
        'itching': request.form['itching'],
        'diet_score': request.form['diet-score'],
        'stress': request.form['stress'],
        'water_intake': request.form['water-intake']
    }

    csv_file = os.path.join(app.root_path, 'form_data.csv')

    if not os.path.exists(csv_file):
        df = pd.DataFrame(columns=data.keys())
        df.to_csv(csv_file, index=False)

    df = pd.DataFrame([data])
    df.to_csv(csv_file, mode='a', index=False, header=False)

    return "<h2 style='text-align:center; color:green;'>Your response has been recorded successfully!</h2>"


if __name__ == '__main__':
    app.run(debug=True)
''''